In [30]:
#bootstrapping 
from numpy import random as rd

os.chdir("/scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/")
!cd /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/
!pwd

/scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap


In [31]:
#deleting all to start a new bootstrap 
!rm -r conn*
!rm -r run*
!rm -r upper*
!rm -r lower*
!rm -r 0*
!rm -r bet*
!rm -r dfe_outfile_*
!rm -r output.txt
!rm -r prop_muts_in_s_ranges.out
!rm -r low_conn*
!rm -r alpha_omega_a_between_centrality.png

rm: cannot remove `conn*': No such file or directory
rm: cannot remove `upper*': No such file or directory
rm: cannot remove `lower*': No such file or directory
rm: cannot remove `0*': No such file or directory
rm: cannot remove `bet*': No such file or directory
rm: cannot remove `output.txt': No such file or directory
rm: cannot remove `prop_muts_in_s_ranges.out': No such file or directory
rm: cannot remove `low_conn*': No such file or directory
rm: cannot remove `alpha_omega_a_between_centrality.png': No such file or directory


In [32]:
import os
for i in range(101):
    os.mkdir("run" + str(i))
    

In [33]:
%matplotlib inline
import pandas as pd
from itertools import product
import pandas as pd
import numpy as np
from collections import Counter
import collections
from numpy import random as rd
from random import sample

#for random connectivity
def make_dfe_input(path):
    os.chdir("/scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/bootstrap/")
    df = pd.read_table('new_485_genes_all_network_metrics.csv', sep=",")
    
    df_random = df.sample(50) #sampling 50 random genes 
    
    df_random_names = df_random['Gene_name'].tolist()
  
    random_lst = (list(rd.choice(df_random_names, size=100, replace=True)))

    random_newDF = pd.DataFrame() #creates a new dataframe that's empty
    for key,value in (dict(Counter(random_lst)).items()):
        for i in range(value):
                random_newDF = random_newDF.append(df_random.loc[df_random['Gene_name'] == key]) # ignoring index is optional


    def sum_SFSs(list_of_SFSs):
        # for a bunch of SFSs that are the same length sum them all up into one big one\
        total_SFS = [sum(i) for i in zip(*list_of_SFSs)]
        return total_SFS

    def column2SFSs(column_of_SFSs):
        list_of_SFSs = []
        for s in list(column_of_SFSs):
            exec("list_of_SFSs.append(" + s + ")")
        return list_of_SFSs

    from ness_vcf import SFS

    total_SFS0_random = SFS(sum_SFSs(column2SFSs(random_newDF.SFS0)))
    total_SFS4_random = SFS(sum_SFSs(column2SFSs(random_newDF.SFS4)))
    SFSs = [total_SFS0_random,total_SFS4_random]

    for s in SFSs:
        s.fold()

    SFS_random_0 = " ".join(str(item) for item in SFSs[0].sfs)
    SFS_random_1 = " ".join(str(item) for item in SFSs[1].sfs)
    !printf "1\n18\n" > $path/SFS.random_connectivity.txt
    !echo $SFS_random_0 >> $path/SFS.random_connectivity.txt
    !echo $SFS_random_1 >> $path/SFS.random_connectivity.txt
    
    random_sites_0 = sum(random_newDF.total_0)
    random_diff_0 = sum(random_newDF.differences_0)
    random_sites_4 = sum(random_newDF.total_4)
    random_diff_4 = sum(random_newDF.differences_4)
    !echo 1 $random_sites_0 $random_diff_0 > $path/divergence.random_connectivity.txt
    !echo 0 $random_sites_4 $random_diff_4 >> $path/divergence.random_connectivity.txt



In [34]:
#writing  to incorrect folder (just dfe not dfe random )
import re
def get_dfe_outputs(filename, t):
    with open(filename) as f:
        content = f.readlines()[0]
        print(content)
        match = re.search('lambda (\d+.\d+) selected_divergence (\d+.\d+) alpha (\d+.\d+) omega_A (\d+.\d+)', content)
        with open(t+'_lambda.txt', 'a') as the_file:
            the_file.write("{}\n".format(match.group(1)))
        with open(t+'_selected_divergence.txt', 'a') as the_file:
            the_file.write("{}\n".format(match.group(2)))
        with open(t+'_alpha.txt', 'a') as the_file:
            the_file.write("{}\n".format(match.group(3)))
        with open(t+'_omega_A.txt', 'a') as the_file:
            the_file.write("{}\n".format(match.group(4)))

In [35]:
import os
rootdir = '/scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap'
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for folder in dirs:
        if folder.startswith("run"):
            new_path = os.path.join(subdir, folder)
#             print("this is the path " + str(new_path))
            os.chdir(new_path) #change directory to new_path 
            if not os.path.exists("random"):
                os.makedirs("random")
            os.chdir(new_path + "/random") #change directory to new_path
            
            if not os.path.exists("random_connectivity_neutral") and not os.path.exists("random_connectivity_selected"):
                os.makedirs("random_connectivity_neutral")
                os.makedirs("random_connectivity_selected")
        
            #send the path random to the function, to enter divergence and diversity )
            make_dfe_input(new_path + "/random")
            
            ##random 
            !echo -e "data_path_1    /scratch/research/tmp_apps/dfe-alpha-release-2.16/data/ \nsfs_input_file    /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/$folder/random/SFS.random_connectivity.txt \nest_dfe_results_dir   /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/$folder/random/random_connectivity_neutral \nsite_class 0 \nfold       1 \nepochs     2 \nsearch_n2       1 \nt2_variable     1 \nt2              50" > /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/$folder/random/run.random.est_dfe-site_class0.config.txt            
            
            
            !bash dfe_first_step.sh /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/$folder/random/run.random.est_dfe-site_class0.config.txt
            
            i+= 1
            print("++++++++++++++++++++++++++++++++++++++*******" + str(i) + "*********************************************************************************" )

            #random class1
            !echo -e "data_path_1    /scratch/research/tmp_apps/dfe-alpha-release-2.16/data/ \nsfs_input_file    /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/$folder/random/SFS.random_connectivity.txt \nest_dfe_results_dir   /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/$folder/random/random_connectivity_selected \nest_dfe_demography_results_file    /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/$folder/random/random_connectivity_neutral/est_dfe.out \nsite_class    1 \nfold    1 \nepochs    2 \nmean_s_variable    1 \nmean_s    -0.1 \nbeta_variable    1 \nbeta    0.5" > /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/$folder/random/run.random.est_dfe-site_class1.config.txt            

            
            !bash dfe_first_step.sh /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/$folder/random/run.random.est_dfe-site_class1.config.txt
            
            !bash dfe_second_step.sh /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/$folder/random/random_connectivity_selected/est_dfe.out ../random_bootstrap/$folder/randomDFE.results.txt
            
            !echo -e "data_path_1    /scratch/research/tmp_apps/dfe-alpha-release-2.16/data/ \ndivergence_file         /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/$folder/random/divergence.random_connectivity.txt \nest_alpha_omega_results_file    /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/$folder/random/est_alpha_omega.out \nest_dfe_results_file     /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/$folder/random/random_connectivity_selected/est_dfe.out \nneut_egf_file           /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/$folder/random/random_connectivity_neutral/neut_egf.out \nsel_egf_file            /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/$folder/random/random_connectivity_selected/sel_egf.out \ndo_jukes_cantor         1 \nremove_poly             1" > /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/$folder/random/run.random.est_alpha_omega.config.txt

            !bash testing.sh /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/$folder/random/run.random.est_alpha_omega.config.txt  
            
            get_dfe_outputs('/scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/' + folder + '/random/est_alpha_omega.out', "random")



ML estimation of the DFE version 2.16
Warning messages are suppressed!!!
Opened file /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/run84/random/run.random.est_dfe-site_class0.config.txt for read.
Read data_path_1 /scratch/research/tmp_apps/dfe-alpha-release-2.16/data/ from /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/run84/random/run.random.est_dfe-site_class0.config.txt
Read sfs_input_file /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/run84/random/SFS.random_connectivity.txt from /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/run84/random/run.random.est_dfe-site_class0.config.txt
Read est_dfe_results_dir /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_bootstrap/run84/random/random_connectivity_neutral from /scratch/research/projects/chlamydomonas/network_evolution/analysis/dfe/random_b